In [5]:
from PCSampler import PCSampler, vis_pc, transform_cloud
from manifpy import SE3, SO3, SE3Tangent
import numpy as np

In [2]:
S = PCSampler()
S.load_samples()
clouds = S.get_clouds()
transforms = S.get_transforms(use_quat=True)

In [45]:
#get the two first clouds and their tranformations to world
c0, c1 = clouds[7], clouds[8]
T0, T1 = SE3(transforms[0]), SE3(transforms[1])
c0_world = transform_cloud(c0, T0.transform())
c1_world = transform_cloud(c1, T1.transform())
vis_pc([c0_world, c1_world])

In [46]:
#get ground truth relative transformation
T = (T1 - T0).exp()
c1_0 = transform_cloud(c1, T.transform())
vis_pc([c1, c1_0])


In [75]:
#give c1 some random perturbations around T 
cov = 0.1
p = np.sqrt(cov)*np.random.normal(size=6)
p[3] = 0
p[4] = 0
T_p = T+SE3Tangent(p)
vis_pc([c1_0, transform_cloud(c1_0, T_p.transform())])

In [60]:

#registering c1 to c0
import open3d as o3d

#convert clouds to open3d clouds
pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(c0)
pcd0.estimate_normals()

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(c1)
pcd0.estimate_normals()

#do registration
treshold = 1
initial_T = np.eye(4)
reg = o3d.pipelines.registration.registration_icp(
    pcd1, pcd0, treshold, initial_T, 
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)
icp_transform = reg.transformation
print(icp_transform)
c1_registered = transform_cloud(c1, icp_transform)

#vis_pc([c0, c1_0])
vis_pc([c0, c1_registered])

[[ 9.91003522e-01  1.33833437e-01 -7.94497496e-04  6.21230412e-02]
 [-1.33834211e-01  9.91003213e-01 -1.01774772e-03 -1.16727915e-01]
 [ 6.51140896e-04  1.11492252e-03  9.99999166e-01  3.68157959e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
